<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Klassiske_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import sklearn.model_selection
from statistics import mean

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
optuna.logging.set_verbosity(optuna.logging.WARNING)

Evalueringsmetrikker

In [4]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

# Egendefinerte moduler

In [5]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [6]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag

In [7]:
def evaluering_beregning(y_test, y_test_prediksjon, y_trening, y_trening_prediksjon):
  rmse_test = rmse(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)
  mae_test = mean_absolute_error(y_test, y_test_prediksjon)
  mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)

  rmse_trening = rmse(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)
  mae_trening = mean_absolute_error(y_trening, y_trening_prediksjon)
  mape_trening = mean_absolute_percentage_error(y_trening, y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [8]:
def evaluering_resultater(detailed_objective):
  rmse_test = detailed_objective(study.best_trial)[0]
  r2_test = detailed_objective(study.best_trial)[1]
  mae_test = detailed_objective(study.best_trial)[2]
  mape_test = detailed_objective(study.best_trial)[3]

  rmse_trening = detailed_objective(study.best_trial)[4]
  r2_trening = detailed_objective(study.best_trial)[5]
  mae_trening = detailed_objective(study.best_trial)[6]
  mape_trening = detailed_objective(study.best_trial)[7]

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [9]:
def rmse_validering_StratifiedKFold(modell, datasett, respons, n_splitt):

  skf = StratifiedKFold(n_splits=n_splitt)

  rmse_validering_resultat = []

  for trening, validering in skf.split(datasett, respons):
    X_trening_fold = (datasett.iloc[trening, :]).iloc[:, :-1]
    X_validering_fold = (datasett.iloc[validering, :]).iloc[:, :-1]

    y_trening_fold = (datasett.iloc[trening, :]).iloc[:, -1]
    y_validering_fold = (datasett.iloc[validering, :]).iloc[:, -1]

    modell.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = modell.predict(X_validering_fold)

    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  return mean(rmse_validering)

# Importering av relevant data

In [10]:
# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
collagen_data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

collagen_data_normal.index = pd.to_datetime(collagen_data_normal.index,
                                            format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [11]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [ ]:
collagen_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatPercent  29136 non-null  float64
 7   NIRfat         29136 non-null  float64
 8   NIRash         29136 non-null  float64
 9   NIRwater       29136 non-null  float64
 10  TT08           29136 non-null  float64
 11  TT20           29136 non-null  float64
 12  TT12           29136 non-null  float64
 13  Collagen       89 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [ ]:
collagen_data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18568 entries, 2022-11-02 00:00:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18568 non-null  int64  
 1   EnzymeType_A2  18568 non-null  int64  
 2   EnzymeType_B   18568 non-null  int64  
 3   EnzymeType_C   18568 non-null  int64  
 4   EnzymeType_D   18568 non-null  int64  
 5   EnzymeType_E   18568 non-null  int64  
 6   RawMatPercent  18568 non-null  float64
 7   NIRfat         18568 non-null  float64
 8   NIRash         18568 non-null  float64
 9   NIRwater       18568 non-null  float64
 10  TT08           18568 non-null  float64
 11  TT20           18568 non-null  float64
 12  TT12           18568 non-null  float64
 13  Collagen       31 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


# NIR målinger inkludert

## Alternativ 1: Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

### Alternativ 1: Et testsett

In [37]:
# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
collagen_trening_markert, collagen_enzymtyper_markert, collagen_test, _, _, _, _\
=trening_testsett_oppdeling_enzym(rå_data,
                                  collagen_data,
                                  test_andel=test_andel)

# Deler datasettene i verdier for forklaringsvariabler og responsvariabler
X_trening_markert = collagen_trening_markert.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]
y_trening_markert = collagen_trening_markert.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

**RandomForestRegressor**

Hyperparamter optimalisering med Optuna

In [38]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  # Manuell Kryssvalidering med Kfold og optimalisering ift. gjennomsnittlig rmse
  splits = 2
  skf = StratifiedKFold(n_splits = splits)

  rmse_validering_resultat = []

  for train, test in skf.split(collagen_trening_markert, collagen_enzymtyper_markert):
    X_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, :-1]
    X_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, :-1]

    y_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, -1]
    y_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, -1]

    rfr_pipeline.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  rmse_validering_snitt = mean(rmse_validering_resultat)

  return rmse_validering_snitt

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-22 14:04:59,439] A new study created in memory with name: no-name-16377ffe-3878-4541-8f99-f7dceb4ec5ea
[I 2024-02-22 14:05:00,246] Trial 0 finished with value: 4.081831491649947 and parameters: {'n_estimators': 184, 'max_depth': 1}. Best is trial 0 with value: 4.081831491649947.
[I 2024-02-22 14:05:00,934] Trial 1 finished with value: 3.8435848818482707 and parameters: {'n_estimators': 181, 'max_depth': 10}. Best is trial 1 with value: 3.8435848818482707.
[I 2024-02-22 14:05:01,490] Trial 2 finished with value: 3.8548686606031817 and parameters: {'n_estimators': 133, 'max_depth': 5}. Best is trial 1 with value: 3.8435848818482707.
[I 2024-02-22 14:05:01,974] Trial 3 finished with value: 4.0842283091322305 and parameters: {'n_estimators': 188, 'max_depth': 1}. Best is trial 1 with value: 3.8435848818482707.
[I 2024-02-22 14:05:02,413] Trial 4 finished with value: 3.827225657385628 and parameters: {'n_estimators': 163, 'max_depth': 7}. Best is trial 4 with value: 3.82722565738

Evaluering av beste modell

In [39]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

# Bruker beste paramtere for studiet for å lage modell
beste_parametere = study.best_params

rfr_pipeline = Pipeline([
    ('skalerer', StandardScaler()),
    ('modell', RandomForestRegressor(**beste_parametere,
                                     random_state=random_seed))
    ])

# Trener modell for videre evaluering
rfr_pipeline.fit(X_trening_markert, y_trening_markert)
y_test_prediksjon = rfr_pipeline.predict(X_test)
y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

# Henter ut evalueringsresulateter av beste modell fra undersøkelse
rmse_test, r2_test, mae_test, mape_test,\
rmse_trening, r2_trening, mae_trening, mape_trening\
= evaluering_beregning(y_test,
                       y_test_prediksjon,
                       y_trening_markert,
                       y_trening_prediksjon)


# Legger resulater til i datasett
resultater = [test_andel,
              rmse_test, r2_test, mae_test, mape_test,
              rmse_trening, r2_trening, mae_trening, mape_trening,
              beste_parametere]

collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

In [40]:
collagen_rfr_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,4.006011,0.538205,3.47979,0.176584,1.232243,0.940077,0.953729,0.046481,"{'n_estimators': 121, 'max_depth': 7}"


### Alternativ 1: Flere testandeler

In [43]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

test_andeler = [0.1, 0.2]

for test_andel in test_andeler:

  print("Test andel:", test_andel)

  collagen_trening_markert, collagen_enzymtyper_markert, collagen_test, _, _, _, _\
  =trening_testsett_oppdeling_enzym(rå_data,
                                    collagen_data,
                                    test_andel=test_andel)

  X_trening = collagen_trening_markert.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]
  y_trening = collagen_trening_markert.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  def objective(trial):
    parametere = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'max_depth': trial.suggest_int('max_depth', 1, 10)
    }
    rfr_pipeline = Pipeline([
        ('skalerer', StandardScaler()),
        ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
    ])

    # Manuell Kryssvalidering med Kfold og optimalisering ift. gjennomsnittlig rmse
    splits = 2
    skf = StratifiedKFold(n_splits = splits)

    rmse_validering_resultat = []

    for train, test in skf.split(collagen_trening_markert, collagen_enzymtyper_markert):
      X_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                      random_state=random_seed))
      ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [test_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

[I 2024-02-22 14:34:28,737] A new study created in memory with name: no-name-20a78db8-42af-44c8-848f-278e3d1befe1


Test andel: 0.1


[I 2024-02-22 14:34:29,276] Trial 0 finished with value: 3.718199457962424 and parameters: {'n_estimators': 191, 'max_depth': 7}. Best is trial 0 with value: 3.718199457962424.
[I 2024-02-22 14:34:29,755] Trial 1 finished with value: 3.7215678854928322 and parameters: {'n_estimators': 170, 'max_depth': 9}. Best is trial 0 with value: 3.718199457962424.
[I 2024-02-22 14:34:30,106] Trial 2 finished with value: 3.708750723674673 and parameters: {'n_estimators': 130, 'max_depth': 10}. Best is trial 2 with value: 3.708750723674673.
[I 2024-02-22 14:34:30,587] Trial 3 finished with value: 3.7640622495395504 and parameters: {'n_estimators': 187, 'max_depth': 2}. Best is trial 2 with value: 3.708750723674673.
[I 2024-02-22 14:34:30,935] Trial 4 finished with value: 3.6983084650892115 and parameters: {'n_estimators': 122, 'max_depth': 8}. Best is trial 4 with value: 3.6983084650892115.
[I 2024-02-22 14:34:31,234] Trial 5 finished with value: 4.0358930809153755 and parameters: {'n_estimators': 1

Test andel: 0.2


[I 2024-02-22 14:35:12,612] Trial 0 finished with value: 3.8716522260348736 and parameters: {'n_estimators': 141, 'max_depth': 6}. Best is trial 0 with value: 3.8716522260348736.
[I 2024-02-22 14:35:13,018] Trial 1 finished with value: 4.080956060246373 and parameters: {'n_estimators': 157, 'max_depth': 1}. Best is trial 0 with value: 3.8716522260348736.
[I 2024-02-22 14:35:13,351] Trial 2 finished with value: 3.8683797613100093 and parameters: {'n_estimators': 125, 'max_depth': 4}. Best is trial 2 with value: 3.8683797613100093.
[I 2024-02-22 14:35:13,751] Trial 3 finished with value: 3.833575045607195 and parameters: {'n_estimators': 143, 'max_depth': 7}. Best is trial 3 with value: 3.833575045607195.
[I 2024-02-22 14:35:14,207] Trial 4 finished with value: 3.936453240190788 and parameters: {'n_estimators': 175, 'max_depth': 2}. Best is trial 3 with value: 3.833575045607195.
[I 2024-02-22 14:35:14,577] Trial 5 finished with value: 3.8415899889504064 and parameters: {'n_estimators': 1

In [ ]:
collagen_rfr_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.10,4.358763,0.504547,3.611236,0.165519,1.221333,0.943056,0.990834,0.046845,"{'n_estimators': 120, 'max_depth': 7}"
1,0.15,3.897934,0.411306,3.368516,0.151419,1.237420,0.944764,0.974233,0.048314,"{'n_estimators': 100, 'max_depth': 8}"
2,0.20,3.990356,0.541807,3.474658,0.176138,1.279863,0.935356,0.991324,0.048822,"{'n_estimators': 177, 'max_depth': 7}"


### Alternativ 1: Flere treningsandeler

In [32]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

collagen_knr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

# Setter fast testandel
test_andel = 0.1

# Deler datasettet i trening og testsett
collagen_trening_markert, collagen_enzymtyper_markert,\
collagen_test, _, collagen_trening_umarkert,\
_, collagen_trening = trening_testsett_oppdeling_enzym(rå_data,
                                                       collagen_data,
                                                       test_andel=test_andel)

# Treningssett
X_test = collagen_test.iloc[:, :-1]
y_test = collagen_test.iloc[:, -1]

**RandomForestRegressor**

In [ ]:
# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

for trenings_andel in trenings_andeler:
  if trenings_andel < 1.0:
      ubenyttet_andel = (1.0-trenings_andel)
      treningssett_markert, enzymtyper_trening_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         collagen_trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = collagen_trening_markert
    enzymtyper_trening_markert = collagen_enzymtyper_markert
    treningssett_umarkert = collagen_trening_umarkert
    treningssett = collagen_trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 10)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
      ])

    splits = 2
    skf = StratifiedKFold(n_splits = splits)

    rmse_validering_resultat = []

    for train, test in skf.split(treningssett_markert, enzymtyper_trening_markert):
      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere, random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

In [ ]:
collagen_rfr_resultater_enzym

**KNeighborsRegressor**

In [ ]:
# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

for trenings_andel in trenings_andeler:
  if trenings_andel < 1.0:
      ubenyttet_andel = (1.0-trenings_andel)
      treningssett_markert, enzymtyper_trening_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         collagen_trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = collagen_trening_markert
    enzymtyper_trening_markert = collagen_enzymtyper_markert
    treningssett_umarkert = collagen_trening_umarkert
    treningssett = collagen_trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 10),
          'p': trial.suggest_int('p', 1, 10)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])

    splits = 2
    skf = StratifiedKFold(n_splits = splits)

    rmse_validering_resultat = []

    for train, test in skf.split(treningssett_markert, enzymtyper_trening_markert):
      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_enzym.loc[len(collagen_knr_resultater_enzym)] = resultater

In [42]:
collagen_knr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.8,4.449539,0.430288,3.578571,0.191793,3.494483,0.544844,2.921205,0.141410,"{'n_neighbors': 7, 'p': 2}"
1,0.9,4.102597,0.515668,3.329815,0.177209,3.532462,0.520610,2.912855,0.141597,"{'n_neighbors': 9, 'p': 2}"
2,1.0,4.008287,0.537680,3.215333,0.169715,3.773886,0.437949,3.146972,0.150234,"{'n_neighbors': 10, 'p': 1}"


## Alternativ 2: Fordelt på dag og kontinuitet

### Alternativ 2: Et testsett

In [ ]:
test_andel = 0.2
collagen_trening, collagen_test, _, _ = trening_testsett_oppdeling_dag(rå_data,
                                                                       collagen_data,
                                                                       test_andel=test_andel)

X_trening = collagen_trening.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]
y_trening = collagen_trening.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

**RandomForestRegressor**

Hyperparameteroptimalisering med Optuna

In [ ]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rfr_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  rmse_test = rmse(y_test, y_test_prediksjon)
  return rmse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

In [ ]:
# Bruker beste paramtere for studiet for å lage modell
beste_parametere = study.best_params

rfr_pipeline = Pipeline([
    ('skalerer', StandardScaler()),
    ('modell', RandomForestRegressor(**beste_parametere, random_state=random_seed))
])

# Trener modell for videre evaluering
rfr_pipeline.fit(X_trening_markert, y_trening_markert)
y_test_prediksjon = rfr_pipeline.predict(X_test)
y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

# Henter ut evalueringsresulateter av beste modell fra undersøkelse
rmse_test, r2_test, mae_test, mape_test,\
rmse_trening, r2_trening, mae_trening, mape_trening\
= evaluering_beregning(y_test,
                        y_test_prediksjon,
                        y_trening_markert,
                        y_trening_prediksjon)


# Legger resulater til i datasett
resultater = [trenings_andel,
              rmse_test, r2_test, mae_test, mape_test,
              rmse_trening, r2_trening, mae_trening, mape_trening,
              beste_parametere]

collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Test andel",
                                                      "RMSE test",
                                                      "R2 test",
                                                      "MAE test",
                                                      "MAPE test",
                                                      "RMSE trening",
                                                      "R2 trening",
                                                      "MAE trening",
                                                      "MAPE trening",
                                                      "Beste parametere"])

collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

In [ ]:
collagen_rfr_resultater_dag

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,2.756939,0.678066,2.543116,0.119563,1.297451,0.939915,1.053718,0.051226,"{'n_estimators': 107, 'max_depth': 10}"


### Alternativ 2: Flere testandeler

In [ ]:
collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Test andel",
                                                      "RMSE test",
                                                      "R2 test",
                                                      "MAE test",
                                                      "MAPE test",
                                                      "RMSE trening",
                                                      "R2 trening",
                                                      "MAE trening",
                                                      "MAPE trening",
                                                      "Beste parametere"])

test_andeler = [0.1, 0.2]

for test_andel in test_andeler:
  collagen_trening, collagen_test, _, _ = trening_testsett_oppdeling_dag(rå_data,
                                                                         collagen_data,
                                                                         test_andel=test_andel)

  X_trening = collagen_trening.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]
  y_trening = collagen_trening.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  def objective(trial):
    parametere = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'max_depth': trial.suggest_int('max_depth', 1, 10)
    }
    rf_pipeline = Pipeline([
        ('skalerer', StandardScaler()),
        ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
    ])

    rf_pipeline.fit(X_trening, y_trening)
    y_test_prediksjon = rf_pipeline.predict(X_test)
    rmse_test = rmse(y_test, y_test_prediksjon)
    return rmse_test

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere, random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                          y_test_prediksjon,
                          y_trening_markert,
                          y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [test_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

In [ ]:
collagen_rfr_resultater_dag

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.1,2.847029,0.535815,2.556092,0.120363,1.372737,0.932632,1.101755,0.053159,"{'n_estimators': 180, 'max_depth': 7}"
1,0.2,2.756939,0.678066,2.543116,0.119563,1.297451,0.939915,1.053718,0.051226,"{'n_estimators': 107, 'max_depth': 10}"


# NIR målinger ekskludert